## Best Time  API Notebook
- This Notebook will:
- Give a brief intro to what Best Time is and why we are going to use it
- Then I will load in data that was returned from a query
- I will clean and process it and send it do a database
- In a seperate notebook I will train a prediction model for this dataset


In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json


### How I think we should Utilize this API:
- Find all the attractions, theatres, bars etc that we want to include in our model
- Format them in an array with the exact name and address (or wont get a match)
- Query BestTime API ( 2 credits for correct query, 1 for a one that they don't have data on)
- Store this data in a database
- Query the database and present the data for the correct location as desired
- Notes:
- There is a lot more data to display if we want
- We should be consesrvative with the API credits
### According to BestTime this is how their data is collected:
- "The data is based on anonymized smartphone GPS signals. Third-party mobile apps collect the data from users who opt-in.
- For competitive reasons we don’t disclose the third-party apps/ and sources. There are some companies active collecting and merging data from different apps. We also have some competitors, but mostly focus on foot traffic data with seasonal/ demographic data. We focus mostly on how busy venues are at certain hours of the week, and we have additional filter tools to e.g. sort venues based on how busy they are per day/ hour etc.
- Privacy related info: The data is aggregated and normalized directly. We don't collect personal data, not even anonymized personal identification numbers. Consequently, we can't link any of our internal data with a single person."


### What values should I extract from the API response:
- Venue Name
- Venue Address
- Venue Latitude
- Venue Longitude
- Venue types (can have multiple types)
- Venue opening and closing hours
- Day of the week (0=Monday, 1=Tuesday, etc.)
- Day hour of the day data (24 values starting at 6AM)
- Day % data (ensure links correctly to the hour and day)

### For sending the API response provide the most accurate address possible and get the correct name and spelling
- Format in a long string with even being names and odd being addresses
- Try 20 venues at a time
- Store in json format and process in a notebook
- Export this as a csv file for use in a PostgreSQL DB



### 2 tables in the database:
- Primary Key is the Venue Name
- One for venues with the following columns:
- Venue Name, Venue Address, Venue Co-ordinates, Venue types (can have multiple types)
- One for the data with the following columns:
- Venue Name, Day Opening Hour, Day Closing Hour,Day of the Week, Hour of the Day, Percentage Busyness

In [4]:
'''
 #plot the time vs busyness:
        plt.figure(figsize=(5, 3))
        #color code the bars based on busyness percentage
        colors = ['green' if p < 40 else 'blue' if p < 70 else 'red' for p in busy_data[i]]
        plt.bar(time_data[i], busy_data[i], color=colors)
        #plot as a coloured bar chart, the higher the number the more red it is
        plt.bar(time_data[i], busy_data[i], color=colors)

        #Add labels
        plt.xlabel('Hour')
        plt.ylabel('% Busyness')
        plt.title('% Busyness vs Time for '+venue_name+ ' on '+ day)
        #sttandardize % to 100 on y axis:
        plt.ylim(0,100)
        #Display the plot
        
        plt.show()
        '''

"\n #plot the time vs busyness:\n        plt.figure(figsize=(5, 3))\n        #color code the bars based on busyness percentage\n        colors = ['green' if p < 40 else 'blue' if p < 70 else 'red' for p in busy_data[i]]\n        plt.bar(time_data[i], busy_data[i], color=colors)\n        #plot as a coloured bar chart, the higher the number the more red it is\n        plt.bar(time_data[i], busy_data[i], color=colors)\n\n        #Add labels\n        plt.xlabel('Hour')\n        plt.ylabel('% Busyness')\n        plt.title('% Busyness vs Time for '+venue_name+ ' on '+ day)\n        #sttandardize % to 100 on y axis:\n        plt.ylim(0,100)\n        #Display the plot\n        \n        plt.show()\n        "

In [6]:
#Extracting data to be stored in json files
# Read the JSON file with the 5 initial attractions responses
with open('okAll.json') as file:
    ok_data = json.load(file)

#number of venues
num=197
# Create an empty DataFrame to store the venue data
venue_data = pd.DataFrame(columns=['Venue Name', 'Venue Address', 'Venue Coordinates', 'Venue Types', 'Average Dwell Time'])
busyness_data = pd.DataFrame(columns=['Venue Name','Opening Hour', 'Closing Hour', 'Day','Hour', 'Busyness'])

#select the first venue
for k in range(0,num):
    data=ok_data[k]

    #populating the venue data array with valid info
    venue_name=data['venue_info']['venue_name']
    venue_address=data['venue_info']['venue_address']
    lat=data['venue_info']['venue_lat']
    lon=data['venue_info']['venue_lon']
    #standardize the venue co-ordinates to have only 4 decimal places
    lon=round(lon,4)
    lat=round(lat,4)
    #format the co-ordinates into a single string
    venue_coord = f"{lat} {lon}"
    #read out the venue types from the inside array
    num_types=len(data['venue_info']['venue_types'])
    for i in range(0,num_types):
        if i==0:
            venue_types=data['venue_info']['venue_types'][i]
        else:
            venue_types=venue_types+", "+data['venue_info']['venue_types'][i]

    venue_avg_dwell_time=data['venue_info']['venue_dwell_time_avg']
    #append the venue data to the DataFrame
    venue_data.loc[k] = [venue_name, venue_address, venue_coord, venue_types, venue_avg_dwell_time]
    
    #Create empty arrays to store data
    time_data = np.zeros((7, 24))
    busy_data = np.zeros((7, 24))
    for i in range(0,7):
        day=data['analysis'][i]['day_info']['day_text']
        opening_time=data['analysis'][i]['day_info']['venue_open']
        closing_time=data['analysis'][i]['day_info']['venue_closed']
        for j in range(0,24):
            if j+6<24:
                hour=j+6
            else:
                hour=(j+6)%24
            
            percentage_busyness=data['analysis'][i]['day_raw'][j]
            busy_data[i,j]=percentage_busyness
            time= hour
            time_data[i,j]=hour
            #append the busyness data to the DataFrame
            busyness_data.loc[k*168+i*24+j] = [venue_name, opening_time, closing_time, day, time, percentage_busyness]

       
#show these tables nicely
venue_data.head(20)

,Venue Name,Venue Address,Venue Coordinates,Venue Types,Average Dwell Time
0,The New York Stock Exchange (NYSE),"11 Wall St New York, NY 10005 United States",40.7069 -74.0113,"stock_broker, financial_institution, tourist_a...",20
1,Bryant Park,"New York, NY 10018 United States",40.7536 -73.9832,"park, tourist_attraction",30
2,Central Park Carousel,"1802 E 65th St New York, NY 10065 United States",40.7699 -73.9753,"amusement_park_ride, historical_landmark, tour...",20
3,Empire State Building,"20 W 34th St. New York, NY 10001 United States",40.7484 -73.9857,"tourist_attraction, historical_landmark, histo...",0
4,The New York Stock Exchange (NYSE),"11 Wall St New York, NY 10005 United States",40.7069 -74.0113,"stock_broker, financial_institution, tourist_a...",20
5,Central Park Carousel,"1802 E 65th St New York, NY 10065 United States",40.7699 -73.9753,"amusement_park_ride, historical_landmark, tour...",20
6,Chelsea Market,"75 9th Ave New York, NY 10011 United States",40.7424 -74.0061,market,0
7,New-York Historical Society,"170 Central Park West New York, NY 10024 Unite...",40.7793 -73.9739,"museum, art_museum, historical_society, histor...",75
8,The Cathedral Church of St. John the Divine,"1047 Amsterdam Ave New York, NY 10025 United S...",40.8038 -73.9619,"episcopal_church, cathedral, church, tourist_a...",0
9,The Met Cloisters,"99 Margaret Corbin Dr New York, NY 10040 Unite...",40.8649 -73.9317,"art_museum, museum, tourist_attraction",60


In [7]:
busyness_data.head(20)

,Venue Name,Opening Hour,Closing Hour,Day,Hour,Busyness
0,The New York Stock Exchange (NYSE),9,16,Monday,6,0
1,The New York Stock Exchange (NYSE),9,16,Monday,7,0
2,The New York Stock Exchange (NYSE),9,16,Monday,8,0
3,The New York Stock Exchange (NYSE),9,16,Monday,9,40
4,The New York Stock Exchange (NYSE),9,16,Monday,10,60
5,The New York Stock Exchange (NYSE),9,16,Monday,11,80
6,The New York Stock Exchange (NYSE),9,16,Monday,12,95
7,The New York Stock Exchange (NYSE),9,16,Monday,13,100
8,The New York Stock Exchange (NYSE),9,16,Monday,14,95
9,The New York Stock Exchange (NYSE),9,16,Monday,15,85


In [8]:
#check for duplicates in the venue data
venue_data_dup=venue_data[venue_data.duplicated(['Venue Name'])]
venue_data_dup.head(20)

,Venue Name,Venue Address,Venue Coordinates,Venue Types,Average Dwell Time
4,The New York Stock Exchange (NYSE),"11 Wall St New York, NY 10005 United States",40.7069 -74.0113,"stock_broker, financial_institution, tourist_a...",20
5,Central Park Carousel,"1802 E 65th St New York, NY 10065 United States",40.7699 -73.9753,"amusement_park_ride, historical_landmark, tour...",20
10,Empire State Building,"20 W 34th St. New York, NY 10001 United States",40.7484 -73.9857,"tourist_attraction, historical_landmark, histo...",0
55,Chelsea Market,"75 9th Ave New York, NY 10011 United States",40.7424 -74.0061,market,0
59,The Met Cloisters,"99 Margaret Corbin Dr New York, NY 10040 Unite...",40.8649 -73.9317,"art_museum, museum, tourist_attraction",60
63,Empire State Building,"20 W 34th St. New York, NY 10001 United States",40.7484 -73.9857,"tourist_attraction, historical_landmark, histo...",0
72,Eden Gallery,"437 Madison Ave New York, NY 10022 United States",40.7576 -73.9755,"art_gallery, art_center, art_dealer",22
119,David Zwirner,"537 W 20th St New York, NY 10011 United States",40.7467 -74.0072,art_gallery,22
137,The New York Stock Exchange (NYSE),"11 Wall St New York, NY 10005 United States",40.7069 -74.0113,"stock_broker, financial_institution, tourist_a...",20
147,Central Park Carousel,"1802 E 65th St New York, NY 10065 United States",40.7699 -73.9753,"amusement_park_ride, historical_landmark, tour...",20


In [9]:
#remove the duplicates
venue_data=venue_data.drop_duplicates(subset=['Venue Name'])


In [10]:
#put this in a new csv file
venue_data.to_csv('venue_data_172_OK.csv', index=False)
#check the length of the data
len(venue_data)

172

In [11]:
len(busyness_data)

33096

In [12]:
#drop the duplicates from the busyness data
busyness_data=busyness_data.drop_duplicates(subset=['Venue Name', 'Opening Hour', 'Closing Hour', 'Day','Hour'])
#put this in a new csv file
busyness_data.to_csv('busyness_data_172_OK.csv', index=False)
#check the length of the data
len(busyness_data)

28920

In [13]:
#load in the error data
with open('errorAll.json') as file:
    error_data = json.load(file)
#remove the duplicate venues

#number of venues
num=902
# Create an empty DataFrame to store the venue data
e_data = pd.DataFrame(columns=['Venue Name'])

#select the first venue
for k in range(0,num):
    data=error_data[k]

    try: 
        #populating the venue data array with valid info
        venue_name=data['venue_info']['venue_name']
        #venue_address=data['venue_info']['venue_address']
        e_data.loc[k] = [venue_name]

    except KeyError:
        #handle the KeyError and continue to the next iteration
        continue

#check for duplicates in the venue data
e_data_dup=e_data[e_data.duplicated(['Venue Name'])]
e_data_dup.head(20)

#save the data to a csv file
e_data.to_csv('venue_data_901_ERROR.csv')



In [16]:
len(e_data)
e_data.head(20)

,Venue Name
0,The Dakota
1,Wellington Hotel
2,Washington Square Hotel
3,The Jane Hotel
4,Astor On The Park
5,The Plaza
6,United Nations Headquarters
7,USS Intrepid
8,Four Points by Sheraton Manhattan
9,"Holiday Inn Manhattan - Financial District, an..."


In [15]:
busyness_data.head(10)

,Venue Name,Opening Hour,Closing Hour,Day,Hour,Busyness
0,The New York Stock Exchange (NYSE),9,16,Monday,6,0
1,The New York Stock Exchange (NYSE),9,16,Monday,7,0
2,The New York Stock Exchange (NYSE),9,16,Monday,8,0
3,The New York Stock Exchange (NYSE),9,16,Monday,9,40
4,The New York Stock Exchange (NYSE),9,16,Monday,10,60
5,The New York Stock Exchange (NYSE),9,16,Monday,11,80
6,The New York Stock Exchange (NYSE),9,16,Monday,12,95
7,The New York Stock Exchange (NYSE),9,16,Monday,13,100
8,The New York Stock Exchange (NYSE),9,16,Monday,14,95
9,The New York Stock Exchange (NYSE),9,16,Monday,15,85


### Conclusion:
- I have 172 venues with data from Best Time
- 901 venues did not have data
- So we can use the 172 and make up for other venues with taxi, subway and weather data